In [1]:
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import KFold

In [2]:
data_merged = []
labels = []

for d_s in ['MTurkSF', 'Wiki-Manual']: # 'britannica', 

    simp_path = f"/workspace/datasets/ds_labels/{d_s}_simp_labels.pkl"        
    src_path = f"/workspace/datasets/ds_labels/{d_s}_src_labels.pkl"  

    simp_labels = pickle.load(open(simp_path, "rb"))
    src_labels = pickle.load(open(src_path, "rb")) 
    
    for entry in simp_labels:
        data_merged.append(entry.tolist())

    for entry in src_labels:
        data_merged.append(entry.tolist())

    curr_lab = [0] * len(simp_labels) + [1] * len(simp_labels)
    labels = labels + curr_lab

X, y = shuffle(data_merged, labels, random_state=42)

In [3]:
clf_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.01, random_state=42)
clf_rf = RandomForestClassifier(random_state=42)

clfs = {'gb': clf_gb, 'rf': clf_rf}

In [4]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for clf in clfs:
    cv_scores = cross_val_score(clfs[clf], X, y, cv=kfold)
    print(clf)
    print(sum(cv_scores)/len(cv_scores))
    print(cv_scores)
    print('___')

gb
0.5199999999999999
[0.50980392 0.54509804 0.53333333 0.51764706 0.51764706 0.50980392
 0.50196078 0.55686275 0.50196078 0.50588235]
___
rf
0.3952941176470588
[0.44313725 0.36078431 0.36470588 0.36862745 0.41568627 0.42745098
 0.4        0.38039216 0.37254902 0.41960784]
___
